#metaBEAT analysis of Illumina seq output for Dynamic_12S

###Mark sure your metaBEAT image version is the latest

`metaBEAT - metaBarcoding and Environmental DNA Analyses tool version: v.0.97.11-global`

Update from your terminal

####check the image, the ID of metaBEAT v.0.97.11 should be `6f6ac6eca734`

sudo docker images
####if not, use the command to update the image from your terminal

sudo docker pull chrishah/metabeat


In [1]:
!pwd

/home/working


##The first step will be to trim/clean our raw Illumina data

Prepare a text file ' Querymap' specifying the samples to be processed including the format and location of the reads.

The below command expects the Illumina data to be present in 2 fastq files (forward and reverse reads) per sample in a directory ./raw_data/. It expects the files to be named 'plateID_L001', followed by 'R1' or 'R2' to identify the forward/reverse read file respectively. 

We need a query map which lists these files along with the primer combinations for each well in each file, and the length of the primer sequence (including heterogeneity spacers, but NOT including any tags) to be trimmed off.

In [2]:
!mkdir trimming

In [3]:
cd trimming/

/home/working/trimming


In [4]:
!mv ../Querymap_demultiplex_trimming.txt ../trimming/

In [5]:
!ls ../trimming/

Querymap_demultiplex_trimming.txt


In [6]:
!head Querymap_demultiplex_trimming.txt

In [7]:
ls ../raw_data/

E1_D0-12S_R1.fastq.gz   E1_D10-12S_R2.fastq.gz  E4_D06-12S_R1.fastq.gz
E1_D0-12S_R2.fastq.gz   E1_D12-12S_R1.fastq.gz  E4_D06-12S_R2.fastq.gz
E1_D02-12S_R1.fastq.gz  E1_D12-12S_R2.fastq.gz  E4_D08-12S_R1.fastq.gz
E1_D02-12S_R2.fastq.gz  E1_D14-12S_R1.fastq.gz  E4_D08-12S_R2.fastq.gz
E1_D04-12S_R1.fastq.gz  E1_D14-12S_R2.fastq.gz  E4_D10-12S_R1.fastq.gz
E1_D04-12S_R2.fastq.gz  E4_D0-12S_R1.fastq.gz   E4_D10-12S_R2.fastq.gz
E1_D06-12S_R1.fastq.gz  E4_D0-12S_R2.fastq.gz   E4_D12-12S_R1.fastq.gz
E1_D06-12S_R2.fastq.gz  E4_D02-12S_R1.fastq.gz  E4_D12-12S_R2.fastq.gz
E1_D08-12S_R1.fastq.gz  E4_D02-12S_R2.fastq.gz  E4_D14-12S_R1.fastq.gz
E1_D08-12S_R2.fastq.gz  E4_D04-12S_R1.fastq.gz  E4_D14-12S_R2.fastq.gz
E1_D10-12S_R1.fastq.gz  E4_D04-12S_R2.fastq.gz


### The current version of metaBEAT is v.0.97.8-global

In [8]:
%%bash

metaBEAT_global.py -h

usage: metaBEAT.py [-h] [-Q <FILE>] [-B <FILE>] [--g_queries <FILE>] [-v] [-s]
                   [-f] [-p] [-k] [-t] [-b] [-m <string>] [-n <INT>] [-E] [-e]
                   [--read_stats_off] [--PCR_primer <FILE>] [--bc_dist <INT>]
                   [--trim_adapter <FILE>] [--trim_qual <INT>] [--phred <INT>]
                   [--trim_window <INT>] [--read_crop <INT>]
                   [--trim_minlength <INT>] [--merge] [--product_length <INT>]
                   [--merged_only] [--forward_only] [--length_filter <INT>]
                   [--length_deviation <FLOAT>] [-R <FILE>] [--gb_out <FILE>]
                   [--rec_check] [--gb_to_taxid <FILE>] [--cluster]
                   [--clust_match <FLOAT>] [--clust_cov <INT>]
                   [--blast_db <PATH>] [--blast_xml <PATH>]
                   [--update_taxonomy] [--taxonomy_db <FILE>]
                   [--min_ident <FLOAT>] [--min_ali_length <FLOAT>]
                   [--bitscore_skim_LCA <FLOAT>] [--bitscore_skim_adju

##Raw data demultiplex and trimming, removal of adapter sequences and merging of readsusing the metaBEAT pipeline.¶

Now we run the trimming and clustering; note that there is no assignment method selected at this stage so metaBEAT will not attempt to make any assignments

In [10]:
%%bash

echo -e "Starttime: $(date)\n"

metaBEAT_global.py \
-Q Querymap_demultiplex_trimming.txt \
--trim_qual 30 \
--trim_minlength 90 \
--merge \
--product_length 110 \
--read_crop 110 \
--forward_only \
--length_filter 100 \
--length_deviation 0.2 \
-m 12S -o Dyn_12S-trim30min90-merge-c3-id1 \
-n 5 -v \
-@ haikuilee@gmail.com &> log_trimming

echo -e "Endtime: $(date)\n"

Starttime: Tue May  8 12:41:03 UTC 2018

Endtime: Tue May  8 15:33:13 UTC 2018



In [11]:
cd ../

/home/working


## The second step is chimera detection

In [12]:
!mkdir chimera_detection

In [13]:
cd chimera_detection/

/home/working/chimera_detection


### Prepare Ref map
    
Convert reference database from Genbank to fasta format to be used in chimera detection.

Prepare Refmap file, i.e. text file that specifies the location and the format of the reference to be used.

The reference sequences in Genbank format should already be present in the `supplementary_data` directory: 

`12S_Fish_SATIVA_cleaned_May_2017.gb`

`Astatotilapia_calliptera.gb` is positive control

In [14]:
!ls ../supplementary_data/reference_DBs/

12S_Fish_SATIVA_cleaned_May_2017.gb  Astatotilapia_calliptera.gb


In [15]:
%%bash

#Write REFmap
for file in $(ls -1 ../supplementary_data/reference_DBs/* | grep ".gb$")
do
      echo -e "$file\tgb"
done > REFmap.txt

In [16]:
!cat REFmap.txt

../supplementary_data/reference_DBs/12S_Fish_SATIVA_cleaned_May_2017.gb	gb
../supplementary_data/reference_DBs/Astatotilapia_calliptera.gb	gb


In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ haikuilee@gmail.com

In [ ]:
!head refs.fasta

In [ ]:
%%bash

echo -e "Starttime: $(date)\n"

for a in $(cut -f 1 ../trimming/Querymap_demultiplex_trimming.txt)
do
    if [ -s ../trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log
        cd ..

    else
        echo -e "$a is empty"
    fi
done

echo -e "Endtime: $(date)\n"

### Last step is taxonomic assignment of reads based on a BLAST - LCA approach using the metaBEAT pipeline.

In [20]:
cd ..

/home/working


In [21]:
!mkdir non-chimeras

In [22]:
cd non-chimeras/

/home/working/non-chimeras


In [23]:
!pwd

/home/working/non-chimeras


__Prepare Querymap and Refmap txt files__

__To skip the `GLOBAL` folder under chimeras detection, add a new line in below command  if [ "$a" != "GLOBAL" ]__

In [24]:
%%bash

#Querymap
for a in $(ls -l ../chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
   if [ "$a" != "GLOBAL" ]
   then
      echo -e "$a-nc\tfasta\t../chimera_detection/$a/$a-nonchimeras.fasta"
   fi
done > Querymap.txt


#Write REFmap
for file in $(ls -1 ../supplementary_data/reference_DBs/* | grep ".gb$")
do
      echo -e "$file\tgb"
done > REFmap.txt


### Sequence clustering and taxonomic assignment via metaBEAT

In [25]:
%%bash

echo -e "Starttime: $(date)\n"

metaBEAT_global.py \
-Q Querymap.txt \
-R REFmap.txt \
--cluster --clust_match 1 --clust_cov 3 \
--blast --min_ident 1.0 \
-m 12S -n 5 \
-E -v \
-@  haikuilee@gmail.com \
-o Dyn_12S-trim_30-merged-nonchimeras-cluster_1c3-blast-min_ident_1 &> log_non-chimeras

echo -e "Endtime: $(date)\n"

Starttime: Tue May  8 21:03:44 UTC 2018

Endtime: Tue May  8 21:16:13 UTC 2018



#### Results are under the GLOBAL folder